<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=860f040c6cb7da2444e38f30acc810fe078cd81cf7c7874fc137b47255580232
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 11:45:15
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 29.33 K (0.21%)
Current PnL: -16.77 L (-11.28%)
CY Booked + Current PnL: -5.33 L (-3.59%)
-------------------
Total profit:  2.03 L
Total loss:  -18.80 L
-------------------
Total Booked + Current PnL: 21.65 L (17.79%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.13 L (57.5%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,104.27,56.0,M-SC,3.98,87973.0,-12804.0,12897.0,-0.77,-12.71,14.66,0.09,245.0,-0.99,0.64,15.58,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-43.66,64.0,H-MC,6.81,112731.0,12227.0,18409.0,-0.26,12.17,16.33,30.49,79.0,0.66,0.82,48.51,MH,ATH,METALS
56,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,220740.0,10374.0,18410.0,0.08,4.93,8.34,13.68,37.0,0.56,1.62,22.27,XY25,NTT,REFINERIES
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201175.0,1037.0,19856.0,0.51,0.52,9.87,10.44,4.0,0.05,1.47,6.11,X40,NTT,FMCG
48,MASFIN,398.61,-16.44,55.0,H-SC,13.00,95175.0,-2805.0,24412.0,0.28,-2.86,25.65,22.05,152.0,-0.11,0.70,38.34,XR,ATH,FINANCE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,61.0,M-MC,6.95,226632.0,1670.0,165985.0,-0.86,0.74,73.24,74.53,192.0,0.01,1.66,33.24,XY24,BTT,STEEL
35,IEX,219.00,-30.16,52.0,H-SC,17.11,199384.0,-570.0,97798.0,-1.02,-0.29,49.05,48.63,115.0,-0.01,1.46,11.23,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.31,47.0,H-MC,2.22,137024.0,1236.0,41518.0,1.03,0.91,30.30,31.48,107.0,0.03,1.00,13.14,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248196.0,465.0,74583.0,-0.16,0.19,30.05,30.30,92.0,0.01,1.82,13.50,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.68,54.0,H-MC,4.20,200760.0,-60.0,68841.0,0.00,-0.03,34.29,34.25,80.0,-0.00,1.47,5.17,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.03,39.0,H-LC,6.38,298775.0,-17067.0,129728.0,-0.58,-5.40,43.42,35.67,5.0,-0.13,2.19,7.70,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.06,46.0,H-LC,7.83,219349.0,-32419.0,85634.0,-0.04,-12.88,39.04,21.13,27.0,-0.38,1.60,17.99,X40,ATH,PAINTS
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201175.0,1037.0,19856.0,0.51,0.52,9.87,10.44,4.0,0.05,1.47,6.11,X40,NTT,FMCG
43,JIOFIN,387.00,0.31,52.0,H-LC,13.23,272577.0,5212.0,60240.0,0.35,1.95,22.10,24.48,48.0,0.09,1.99,57.99,XY24,BTT,FINANCE
79,UNITDSPR,1644.00,-12.80,53.0,H-LC,7.79,228760.0,-5384.0,54010.0,-0.53,-2.30,23.61,20.77,86.0,-0.10,1.67,4.46,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,1.16,72.0,H-LC,1.30,250830.0,4679.0,74522.0,2.78,1.90,29.71,32.18,16.0,0.06,1.84,38.63,X200,ATH,IT
41,ITC,452.0,-37.50,51.0,H-LC,2.52,201175.0,1037.0,19856.0,0.51,0.52,9.87,10.44,4.0,0.05,1.47,6.11,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.22,58.0,H-LC,4.22,282391.0,16965.0,39817.0,0.21,6.39,14.10,21.40,11.0,0.43,2.07,13.94,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,220740.0,10374.0,18410.0,0.08,4.93,8.34,13.68,37.0,0.56,1.62,22.27,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,53.0,H-LC,5.77,214665.0,10165.0,31255.0,0.47,4.97,14.56,20.25,10.0,0.33,1.57,14.14,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,1.16,72.0,H-LC,1.30,250830.0,4679.0,74522.0,2.78,1.90,29.71,32.18,16.0,0.06,1.84,38.63,X200,ATH,IT
7,ATULAUTO,844.00,3948.65,65.0,M-SC,5.79,126260.0,-21108.0,72928.0,2.17,-14.32,57.76,35.16,236.0,-0.29,0.92,28.96,XY24,NTT,AUTO
10,BANDHANBNK,400.00,-10.58,53.0,H-SC,5.13,224504.0,-54055.0,311499.0,1.98,-19.41,138.75,92.42,151.0,-0.17,1.64,29.75,XY24,NTT,BANKS
46,LAOPALA,464.00,116.69,53.0,H-SC,2.36,99193.0,-31912.0,83163.0,1.93,-24.34,83.84,39.09,142.0,-0.38,0.73,27.75,AR,NTT,CERAMICS
15,BSOFT,836.99,-12.08,57.0,H-SC,15.03,98685.0,-41968.0,114751.0,1.57,-29.84,116.28,51.74,131.0,-0.37,0.72,12.54,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJOOENG,143.33,-36.44,37.0,H-SC,11.60,96400.0,-6100.0,46928.0,-1.63,-5.95,48.68,39.83,114.0,-0.13,0.71,0.76,AR,ATH,MISC
31,HINDZINC,730.22,20.84,59.0,M-LC,1.35,197316.0,-7760.0,120323.0,-1.27,-3.78,60.98,54.89,52.0,-0.06,1.44,19.40,X5K,ATH,METALS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89074.0,-11889.0,106417.0,-1.07,-11.78,119.47,93.63,148.0,-0.11,0.65,32.67,SR,ATH,CHEMICALS
35,IEX,219.00,-30.16,52.0,H-SC,17.11,199384.0,-570.0,97798.0,-1.02,-0.29,49.05,48.63,115.0,-0.01,1.46,11.23,XR,NTT,MISC
45,KPIGREEN,731.05,2.11,42.0,H-SC,10.01,121514.0,-3783.0,62713.0,-1.00,-3.02,51.61,47.03,141.0,-0.06,0.89,52.15,MH,ATH,POWER


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,104.27,56.0,M-SC,3.98,87973.0,-12804.0,12897.0,-0.77,-12.71,14.66,0.09,245.0,-0.99,0.64,15.58,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,96.03,56.0,H-SC,9.53,123984.0,-18495.0,38013.0,0.37,-12.98,30.66,13.70,163.0,-0.49,0.91,46.34,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,64.0,H-SC,1.40,227232.0,-42435.0,78759.0,1.20,-15.74,34.66,13.47,138.0,-0.54,1.66,14.93,XY24,NTT,PAINTS
18,CERA,9475.0,-19.33,50.0,H-SC,2.08,146855.0,-29048.0,71063.0,0.17,-16.51,48.39,23.89,149.0,-0.41,1.07,26.83,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.67,108424.0,-23411.0,65640.0,0.41,-17.76,60.54,32.03,98.0,-0.36,0.79,21.44,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,60.0,H-LC,12.33,298572.0,-47384.0,122235.0,0.11,-13.70,40.94,21.64,1.0,-0.39,2.18,5.77,X40,ATH,IT
39,INFY,2275.00,-15.65,59.0,H-LC,8.40,328978.0,16090.0,155607.0,1.45,5.14,47.30,54.87,3.0,0.10,2.41,12.13,X40,BTT,IT
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201175.0,1037.0,19856.0,0.51,0.52,9.87,10.44,4.0,0.05,1.47,6.11,X40,NTT,FMCG
82,VBL,671.64,-17.03,39.0,H-LC,6.38,298775.0,-17067.0,129728.0,-0.58,-5.40,43.42,35.67,5.0,-0.13,2.19,7.70,X40N,ATH,FMCG
1,ABB,7934.00,-37.82,67.0,H-LC,10.36,259728.0,-1891.0,121111.0,0.47,-0.72,46.63,45.57,7.0,-0.02,1.90,10.52,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,61.0,M-MC,6.95,226632.0,1670.0,165985.0,-0.86,0.74,73.24,74.53,192.0,0.01,1.66,33.24,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,13.92,65.0,H-MC,3.57,196540.0,31870.0,36104.0,1.30,19.35,18.37,41.28,89.0,0.88,1.44,28.88,X40N,ATH,AC
83,VOLTAS,1918.49,0.74,57.0,H-MC,2.48,211455.0,19713.0,76314.0,-0.16,10.28,36.09,50.08,99.0,0.26,1.55,17.78,XY25,ATH,AC
17,CAMS,5211.76,0.16,66.0,H-SC,3.82,113792.0,11788.0,32135.0,1.25,11.56,28.24,43.06,122.0,0.37,0.83,30.65,X40N,ATH,MISC
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248196.0,465.0,74583.0,-0.16,0.19,30.05,30.30,92.0,0.01,1.82,13.50,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,64.0,H-MC,6.81,112731.0,12227.0,18409.0,-0.26,12.17,16.33,30.49,79.0,0.66,0.82,48.51,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.66,51.0,M-SC,0.48,100208.0,8710.0,70045.0,-0.41,9.52,69.90,86.07,223.0,0.12,0.73,45.52,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.03,41.0,H-SC,11.66,134079.0,10743.0,123017.0,-0.82,8.71,91.75,108.45,119.0,0.09,0.98,33.01,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159092.0,8147.0,101310.0,0.96,5.40,63.68,72.51,53.0,0.08,1.16,11.74,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,65.0,H-MC,6.13,183729.0,3921.0,108492.0,-0.15,2.18,59.05,62.52,88.0,0.04,1.34,37.01,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1893.69,67.0,L-SC,3.05,54397.0,-82780.0,82874.0,0.22,-60.35,152.35,0.07,267.0,-1.00,0.40,35.29,OX40N,NTT,JEWELLERY
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.42,86208.0,-27341.0,67389.0,1.29,-24.08,78.17,35.27,268.0,-0.41,0.63,111.00,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89074.0,-11889.0,106417.0,-1.07,-11.78,119.47,93.63,148.0,-0.11,0.65,32.67,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,55.0,H-SC,13.00,95175.0,-2805.0,24412.0,0.28,-2.86,25.65,22.05,152.0,-0.11,0.70,38.34,XR,ATH,FINANCE
49,NATIONALUM,247.44,-43.66,64.0,H-MC,6.81,112731.0,12227.0,18409.0,-0.26,12.17,16.33,30.49,79.0,0.66,0.82,48.51,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3948.65,65.0,M-SC,5.79,126260.0,-21108.0,72928.0,2.17,-14.32,57.76,35.16,236.0,-0.29,0.92,28.96,XY24,NTT,AUTO
21,COFFEEDAY,80.0,-41.65,53.0,L-SC,42.42,86208.0,-27341.0,67389.0,1.29,-24.08,78.17,35.27,268.0,-0.41,0.63,111.00,XR,NTT,HOTELS
80,VAIBHAVGBL,521.0,66.94,72.0,H-SC,7.21,144948.0,-37827.0,148891.0,1.51,-20.70,102.72,60.77,125.0,-0.25,1.06,29.82,XR,NTT,JEWELLERY
60,SAMMAANCAP,326.0,-171.59,62.0,M-SC,2.62,82236.0,-19974.0,113362.0,1.47,-19.54,137.85,91.37,208.0,-0.18,0.60,33.50,XY25,NTT,FINANCE
74,TITAGARH,1548.0,1.18,69.0,H-SC,3.26,165900.0,-26115.0,104998.0,0.62,-13.60,63.29,41.08,158.0,-0.25,1.21,40.58,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.12
1,25,43.88
2,50,74.46


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.56
LC    32.16
MC    23.25
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.31
X40      15.43
X40N     11.94
XR       11.76
AR        8.76
XY25      8.26
OX40N     8.22
X200      1.84
MH        1.71
X5K       1.44
SR        1.30
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.35
H-LC    25.44
H-MC    20.29
M-SC    13.64
M-LC     5.65
M-MC     2.65
L-SC     1.57
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.69,-12.29,73.52
FMCG,11.89,-4.30,40.79
FINANCE,9.78,-15.15,60.26
MISC,7.48,-12.42,74.94
BANKS,6.33,-15.40,76.11
PAINTS,5.87,-14.91,32.04
ELECTRICAL,5.76,-4.57,62.20
AC,3.71,4.40,27.98
INSURANCE,3.43,-7.21,43.99


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2839527.0,22
XR,1301063.0,14
AR,1300444.0,9
X40,738320.0,10
X40N,596803.0,9
OX40N,521814.0,10
XY25,395355.0,6
SR,243964.0,2
X5K,120323.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3400487.0,29
M-SC,1379073.0,17
H-LC,1215261.0,14
H-MC,1180401.0,15
M-LC,377296.0,4
M-MC,316259.0,2
L-SC,246410.0,3
L-MC,60494.0,1
L-LC,37576.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1020970.0      6
M-SC       XY24       795462.0      7
H-SC       AR         786720.0      5
           XR         752674.0      7
H-MC       XY24       490933.0      4
H-LC       X40        454587.0      5
           AR         352692.0      2
M-MC       XY24       316259.0      2
H-SC       X40N       284860.0      4
           OX40N      248586.0      4
           SR         243964.0      2
H-LC       X40N       214993.0      3
H-MC       X40        213787.0      4
           XY25       186190.0      2
L-SC       XR         163536.0      2
M-SC       AR         161032.0      2
M-LC       XY24       155663.0      2
M-SC       OX40N      124714.0      4
M-LC       X5K        120323.0      1
M-SC       XR         114557.0      2
           XY25       113362.0      1
H-MC       XR         108492.0      1
M-LC       XR         101310.0      1
H-MC       X40N        96950.0      2
L-SC       OX40N       82874.0      1
H-LC       X200        74522.0      1
M-SC       X40         69946.0      1
H-MC       OX40N       65640.0      1
H-SC       MH          62713.0      1
L-MC       XR          60494.0      1
H-LC       XY24        60240.0      1
           XY25        58227.0      2
L-LC       XY25        37576.0      1
H-MC       MH          18409.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
